In [19]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time

# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 5
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

start_time = time.time()
training_time = time.time() - start_time
print("Training time:", training_time, "seconds")

# Function to preprocess user input
def preprocess_input(user_input):
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)

# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "POSITIVE!"
    else:
        return "NEGATIVE!"
    
# User Interface
print("Welcome to Movie Review Sentiment Analyzer!")

while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)
        


Epoch 1/5
782/782 [==============================] - 91s 111ms/step - loss: 0.4245 - accuracy: 0.8042 - val_loss: 0.3494 - val_accuracy: 0.8466
Epoch 2/5
782/782 [==============================] - 87s 111ms/step - loss: 0.3016 - accuracy: 0.8741 - val_loss: 0.3563 - val_accuracy: 0.8495
Epoch 3/5
782/782 [==============================] - 87s 112ms/step - loss: 0.2488 - accuracy: 0.8976 - val_loss: 0.3572 - val_accuracy: 0.8497
Epoch 4/5
782/782 [==============================] - 86s 110ms/step - loss: 0.2057 - accuracy: 0.9186 - val_loss: 0.3871 - val_accuracy: 0.8464
Epoch 5/5
782/782 [==============================] - 85s 109ms/step - loss: 0.1805 - accuracy: 0.9298 - val_loss: 0.4287 - val_accuracy: 0.8403
Training time: 0.0 seconds
Welcome to Movie Review Sentiment Analyzer!
Enter your movie review (or type 'quit' to exit): bad movie
1/1 [==============================] - 0s 289ms/step
Predicted sentiment: NEGATIVE!
Enter your movie review (or type 'quit' to exit): quit
Thank you 